# Instalando as bibliotecas necessarias


In [ ]:
import requests as rq
import pathlib as path
import pandas as pd
import json
import shutil
import urllib.request
import os
import zipfile
import time
from IPython.display import Image
from IPython.display import clear_output
import io

%matplotlib inline

url_api = "https://localhost:44346/v1/Demo/"
#url_api = "https://localhost:5001/v1/Demo/"

# DOWNLOAD DAS PARTIDAS

Será feito o download de mais de 450 demos e armazenadas em um json para maior facilidade de manuseio

## Agora vamos consumir a API e seus métodos

### Metodo GetPlayers, retorna todos os dados dos jogadores das demos baixadas acima

In [ ]:
index = 1;
start = time.time()

print("Baixando todas partidas demo...")

for link in ligas['demo']:
    filedata = urllib.request.urlopen(link)
    data = filedata.read()
    start = time.time()
    mypath = "partidas"
    
    if not os.path.exists(mypath):
        os.makedirs(mypath, 0o0755)
        print("Pasta criada.")
    
    fileName = mypath + "/" + "partida" + str(index) + ".rar"
    
    with open(fileName, 'wb') as f:
        f.write(data)
        
    with zipfile.ZipFile(fileName) as zip_ref:
        zip_ref.extractall('partidas')
        
    os.remove(fileName)
    
    end = time.time()
    tempoDecorrido = end - start
    print(str(fileName) + " baixada em " + str(tempoDecorrido) + "s")
    
    index+=1;
    

end = time.time()
tempoDecorrido = end - start
print("Todas as partidas baixadas em " + str(tempoDecorrido) + "s")

## Agora vamos consumir a API e seus métodos

### Metodo GetPlayers, retorna todos os dados dos jogadores das demos baixadas acima

In [ ]:
def GetNamesWeapon(data):
    result = ['Name', 'TeamName', 'Killed', 'Death', 'ADR', 'TotalDamageDealt', 'RoundMVPs', 'FlashAssists', 'BombsPlanted', 'Clutches', 'FirstKills', 'FirstDeaths', 'FlashedEnemies', 'WalkQuantityAsTR',
              'DistanceTraveledAsTR', 'DistanceTraveledAsCT']
    for player in data: 
        for weapon in player['Weapons']:
            if weapon['NameWeapon'] not in result:
                result.append(weapon['NameWeapon'])
    return result
                  
def GetInfoWeaponPlayer(name, weapons, jsonResult):
    print()
    dic = {weapon['NameWeapon']: int(weapon['KillQuantity']) for weapon in weapons }
    dic['Name']                 = name
    dic['TeamName']             = jsonResult['TeamName']
    dic['Killed']               = jsonResult['Killed']
    dic['Death']                = jsonResult['Death']
    dic['ADR']                  = jsonResult['ADR']
    dic['TotalDamageDealt']     = jsonResult['TotalDamageDealt']
    dic['RoundMVPs']            = jsonResult['RoundMVPs']
    dic['FlashAssists']         = jsonResult['FlashAssists']
    dic['BombsPlanted']         = jsonResult['BombsPlanted']
    dic['Clutches']             = jsonResult['Clutches']
    dic['FirstKills']           = jsonResult['FirstKills']
    dic['FirstDeaths']          = jsonResult['FirstDeaths']
    dic['FlashedEnemies']       = jsonResult['FlashedEnemies']
    dic['WalkQuantityAsTR']     = jsonResult['WalkQuantityAsTR']
    dic['DistanceTraveledAsTR'] = jsonResult['DistanceTraveledAsTR']
    dic['DistanceTraveledAsCT'] = jsonResult['DistanceTraveledAsCT']## Agora vamos consumir a API e seus métodos

### Metodo GetPlayers, retorna todos os dados dos jogadores das demos baixadas acima
    return dic

In [ ]:
jsonResult = json.loads(rq.get(url_api + "GetPlayers", verify=False).content)

playerWeaponsArray = []

#pego o nome de todas as armas para fazer de colunas
weaponsArray = GetNamesWeapon(jsonResult)
for player in jsonResult:
    playerWeaponsArray.append(GetInfoWeaponPlayer(player['Name'], player['Weapons'], player))

df = pd.DataFrame(data=playerWeaponsArray, columns=weaponsArray)
df = df.fillna(0)
df.to_csv("LiquidVPPlayersStats.csv")
df.head()

In [ ]:
jsonResult = rq.get(url_api + "GetPlayers", verify=False).content
df = pd.read_json(jsonResult)
df.drop(['Weapons'], axis=1, inplace=True)

In [ ]:
df.to_csv("AllPlayersStats.csv")
df

## Filtrando dados

In [ ]:
roles = pd.read_csv("PlayersRoles180.csv")
#roles = pd.read_csv("rolesTest.csv")
roles.info()

In [ ]:
players = pd.read_csv("AllPlayersStats2.csv")
players.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
players.insert(1, 'Role', "")

cols = players.columns.tolist()
playersRolesDF = pd.DataFrame(columns=cols)

players = players.fillna(0)
players.head()

In [ ]:
for index, row in roles.iterrows():
    player = players.loc[players['Name'] == row['Name']]
    if (row['Role'] != 'IGL'):
        player['Role'] = row['Role']
        playersRolesDF = pd.concat([playersRolesDF, player])

playersRolesDF.head()

In [ ]:
#playersRolesDF.drop(['ID', 'Name', 'TeamName', 'DistanceTraveledAsCT', 'Knife', 'P250', 'Desconhecido', 'DualBarettas', 'Swag7', 'Zeus', 'P90', 'XM1014', 'MP7', 'MP5SD', 'Nova', 'M249', 'World', 'Scar20', 'Negev', 'Bizon', 'SawedOff', 'G3SG1', 'P2000'], axis=1, inplace=True)
#playersRolesDF.drop(['ID', 'Name', 'TeamName', 'DistanceTraveledAsCT', 'DistanceTraveledAsTR', 'WalkQuantityAsTR', 'Knife'], axis=1, inplace=True)
playersRolesDF.drop(['ID', 'Name', 'TeamName', 'WalkQuantityAsCT', 'DistanceTraveledAsCT'], axis=1, inplace=True)

In [ ]:
#dataFrame = playersRolesDF.drop('Role', axis=1)
dataframe = playersRolesDF.round(decimals=2)
#dataframe.to_csv('playersRolesDF3.csv')
dataframe

## Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import tree
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle

%matplotlib inline

In [ ]:
score = 0
while (score <= 0.6):
    X_train, X_test, y_train, y_test = train_test_split(dataFrame.drop('Role', axis=1), dataFrame['Role'])
    model = LogisticRegression(multi_class='multinomial', max_iter=10000000000000)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    score = model.score(X_test, y_test)
    clear_output(wait=True)

print(classification_report(y_test, predictions))
sns.heatmap(confusion_matrix(y_test, predictions), annot=True)

### Árvore de decisão

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(4,4), dpi=300)

tree.plot_tree(clf, filled=True)
fig.savefig('DecisionTree.png')

### Salvar o modelo

In [ ]:
fileName = 'models/model.pkl'
with open(fileName, 'wb') as file:
    pickle.dump(model, file)

### Predict one player

In [ ]:
casual_players = pd.read_csv('MyDemoStats.csv')
casual_players.head()

In [ ]:
casual_players.drop(['Unnamed: 0', 'Name', 'TeamName', 'WalkQuantityAsCT', 'DistanceTraveledAsCT'], axis=1, inplace=True)

In [ ]:
casual_players = casual_players.round(decimals=2)
casual_players.insert(0, 'Role', '')
casual_players = casual_players.fillna(0)
casual_players

### Adiciona colunas faltantes comparado ao modelo de treino para que seja possivel fazer a predição

In [ ]:
allPlayers_cols = playersRolesDF.columns.to_list()
casual_players_cols = casual_players.columns.to_list()

for i in range(len(playersRolesDF.columns.to_list())):
    if allPlayers_cols[i] in casual_players.columns:
        continue
    else:
        casual_players.insert(i, allPlayers_cols[i], 0)

#casual_players.drop(['Unnamed: 0', 'Desconhecido', 'Name', 'TeamName', 'DistanceTraveledAsCT', 'WalkQuantityAsTR', 'DistanceTraveledAsTR', 'Knife'], axis=1, inplace=True)
casual_players.drop(['Unnamed: 0', 'Desconhecido', 'Name', 'TeamName', 'DistanceTraveledAsCT', 'WalkQuantityAsTR', 'DistanceTraveledAsTR'], axis=1, inplace=True)
#casual_players.drop(['Desconhecido', 'Name', 'TeamName', 'DistanceTraveledAsCT'], axis=1, inplace=True)
casual_players.head()        

In [ ]:
casual_players_count = len(casual_players.columns)
playersRoles_count = len(playersRolesDF.columns)
print(casual_players_count)
print(casual_players.columns)
print(playersRoles_count)
print(playersRolesDF.columns)

if (casual_players_count < playersRoles_count):
    for i in range(playersRoles_count - casual_players_count):
        casual_players.insert(casual_players_count, i, 0)
        casual_players_count = len(casual_players.columns)

print("===================================================================================")
print(casual_players_count)
print(casual_players.columns)
print(playersRoles_count)
print(playersRolesDF.columns)

In [ ]:
#casual_players.drop(['Unnamed: 0'], axis=1, inplace=True)
casual_players = casual_players.fillna(0)
casual_players.head()

In [ ]:
score = model.score(X_test, y_test)
print(score)

In [ ]:
predictions_proba = model.predict_proba(casual_players.drop('Role', axis=1))
predictions = model.predict(casual_players.drop('Role', axis=1))
#predictions_proba = loaded_model.predict_proba(X_test)
#predictions = loaded_model.predict(X_test)

np.set_printoptions(suppress=True)
print(predictions_proba)
print(predictions)

### Carregando modelo salvo

In [ ]:
with open('models/model60.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
    
predictions_proba = loaded_model.predict_proba(casual_players.drop('Role', axis=1))
predictions = loaded_model.predict(casual_players.drop('Role', axis=1))
#predictions_proba = loaded_model.predict_proba(X_test)
#predictions = loaded_model.predict(X_test)

np.set_printoptions(suppress=True)

print(predictions_proba)
print(predictions)

In [ ]:
print(classification_report(y_test, predictions))
sns.heatmap(confusion_matrix(y_test, predictions), annot=True)

### Metodo GetHeatMap, retorna um mapa de calor da dust2 onde é possível ver onde os jogadores tiveram uma interação e onde eles foram mortos

In [ ]:
response = rq.get(url_api + "GetHeatMap", verify=False, stream=True)

with open('heat_map.png', 'wb') as image:
    shutil.copyfileobj(response.raw, image)
del response

Image(filename='heat_map.png')